In [3]:
!pip install sqlalchemy pymysql pandas

In [ ]:
from sqlalchemy import create_engine
import pandas as pd

USER = ""
PASSWORD = "" 
HOST = "127.0.0.1"
PORT = 3306
DB = "sakila"

engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB}")

In [6]:
pd.read_sql("SELECT 1;", engine)

,1
0,1


In [7]:
def rentals_month(engine, month: int, year: int) -> pd.DataFrame:
    """
    Return rental records for a given month/year from sakila.rental as a DataFrame.
    """
    query = """
        SELECT rental_id, rental_date, inventory_id, customer_id, staff_id, return_date
        FROM rental
        WHERE MONTH(rental_date) = %(month)s
          AND YEAR(rental_date) = %(year)s;
    """
    return pd.read_sql(query, engine, params={"month": month, "year": year})

In [14]:
may_df = rentals_month(engine, 5, 2005)
june_df = rentals_month(engine, 6, 2005)

may_df.head(), june_df.head()

(   rental_id         rental_date  inventory_id  customer_id  staff_id  \
 0          1 2005-05-24 22:53:30           367          130         1   
 1          2 2005-05-24 22:54:33          1525          459         1   
 2          3 2005-05-24 23:03:39          1711          408         1   
 3          4 2005-05-24 23:04:41          2452          333         2   
 4          5 2005-05-24 23:05:21          2079          222         1   
 
           return_date  
 0 2005-05-26 22:04:30  
 1 2005-05-28 19:40:33  
 2 2005-06-01 22:12:39  
 3 2005-06-03 01:43:41  
 4 2005-06-02 04:33:21  ,
    rental_id         rental_date  inventory_id  customer_id  staff_id  \
 0       1158 2005-06-14 22:53:33          1632          416         2   
 1       1159 2005-06-14 22:55:13          4395          516         1   
 2       1160 2005-06-14 23:00:34          2795          239         2   
 3       1161 2005-06-14 23:07:08          1690          285         1   
 4       1162 2005-06-14 23:09:38

In [15]:
def rental_count_month(rentals_df: pd.DataFrame, month: int, year: int) -> pd.DataFrame:
    """
    Given rentals dataframe for a month/year, return a dataframe:
    customer_id | rentals_MM_YYYY
    """
    col_name = f"rentals_{month:02d}_{year}"
    
    counts = (
        rentals_df.groupby("customer_id")
        .size()
        .reset_index(name=col_name)
    )
    return counts

In [16]:
may_counts = rental_count_month(may_df, 5, 2005)
june_counts = rental_count_month(june_df, 6, 2005)

may_counts.head(), june_counts.head()

(   customer_id  rentals_05_2005
 0            1                2
 1            2                1
 2            3                2
 3            5                3
 4            6                3,
    customer_id  rentals_06_2005
 0            1                7
 1            2                1
 2            3                4
 3            4                6
 4            5                5)

In [17]:
def compare_rentals(counts_df_1: pd.DataFrame, counts_df_2: pd.DataFrame) -> pd.DataFrame:
    """
    Join two monthly rental count dataframes on customer_id and add a difference column:
    difference = month2 - month1
    """
    # Identify the month columns (everything except customer_id)
    month_col_1 = [c for c in counts_df_1.columns if c != "customer_id"][0]
    month_col_2 = [c for c in counts_df_2.columns if c != "customer_id"][0]

    merged = counts_df_1.merge(counts_df_2, on="customer_id", how="inner")
    merged["difference"] = merged[month_col_2] - merged[month_col_1]
    return merged

In [18]:
comparison = compare_rentals(may_counts, june_counts)
comparison.head()

,customer_id,rentals_05_2005,rentals_06_2005,difference
0,1,2,7,5
1,2,1,1,0
2,3,2,4,2
3,5,3,5,2
4,6,3,4,1


In [19]:
customer_info = pd.read_sql(
    "SELECT customer_id, first_name, last_name, email FROM customer;",
    engine
)

final = comparison.merge(customer_info, on="customer_id", how="left")
final = final[["customer_id", "first_name", "last_name", "email"] + 
              [c for c in comparison.columns if c != "customer_id"]]

final.head()

,customer_id,first_name,last_name,email,rentals_05_2005,rentals_06_2005,difference
0,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,2,7,5
1,2,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,1,1,0
2,3,LINDA,WILLIAMS,LINDA.WILLIAMS@sakilacustomer.org,2,4,2
3,5,ELIZABETH,BROWN,ELIZABETH.BROWN@sakilacustomer.org,3,5,2
4,6,JENNIFER,DAVIS,JENNIFER.DAVIS@sakilacustomer.org,3,4,1
